In [1]:
pip install uv


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
uv --version

uv 0.9.5
Note: you may need to restart the kernel to use updated packages.


In [3]:
!mkdir uv_homework

mkdir: cannot create directory ‘uv_homework’: File exists


In [4]:
%cd uv_homework

/workspaces/machine-learning-zoomcamp-homework/05-deployment/uv_homework


In [5]:
!uv init

error: Project is already initialized in `/workspaces/machine-learning-zoomcamp-homework/05-deployment/uv_homework` (`pyproject.toml` file exists)


In [6]:
!uv add scikit-learn==1.6.1

Resolved 6 packages in 0.79ms
Audited 5 packages in 0.13ms


In [7]:
!ls

README.md    main.py	      pipeline_v1.bin.1  pyproject.toml
__pycache__  pipeline_v1.bin  pipeline_v1.bin.2  uv.lock


In [8]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-10-27 18:05:43--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-27 18:05:44--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin.3’

pipeline_v1.b

In [9]:
import pickle

with open('pipeline_v1.bin', 'rb') as f:
    model = pickle.load(f)

X_test = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

proba = model.predict_proba([X_test])[0, 1] 

print(f"Probability of conversion: {proba:.3f}")

Probability of conversion: 0.534


/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. Th

In [10]:
pip install fastapi uvicorn


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
from fastapi import FastAPI
from pydantic import BaseModel

with open('pipeline_v1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

app = FastAPI()

@app.post("/")
def predict(client: Client):
    X = [client.dict()]
    proba = model.predict_proba(X)[0, 1]
    return {"subscription_probability": proba}

In [ ]:
!uvicorn main:app --reload --port 8003

INFO:     Will watch for changes in these directories: ['/workspaces/machine-learning-zoomcamp-homework/05-deployment/uv_homework']
INFO:     Uvicorn running on http://127.0.0.1:8003 (Press CTRL+C to quit)
INFO:     Started reloader process [39676] using StatReload
INFO:     Started server process [39678]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
